# ResNet50

## We will use the pretrained ResNet50 model

## imports

In [2]:
import sys

import tensorflow as tf
import tensorflow.keras.layers as tfl

sys.path.append('../')
import utils

## Load data

In [4]:
data = utils.get_data('../data/data.pkl')
X_train = tf.cast(data['X_train'], dtype=tf.float32)
y_train = tf.cast(data['y_train'], dtype=tf.float32)
X_valid = tf.cast(data['X_valid'], dtype=tf.float32)
y_valid = tf.cast(data['y_valid'], dtype=tf.float32)

## Base model

In [5]:
base_model = tf.keras.applications.resnet50.ResNet50(include_top=False,
                                                     weights='imagenet', 
                                                     input_shape=(100, 100, 3))

94773248/94765736 [==============================] - 37s 0us/step


## Digits recog model

In [8]:
def digits_model():
    base_model = tf.keras.applications.resnet50.ResNet50(include_top=False,
                                                     weights='imagenet', 
                                                     input_shape=(100, 100, 3))
    preprocess_input = tf.keras.applications.resnet50.preprocess_input                

    base_model.trainable = False

    inputs = tfl.Input(shape=(100, 100, 3))
    x = preprocess_input(inputs)
    x = base_model(x, training=False)
    x = tfl.GlobalAveragePooling2D()(x)
    x = tfl.Dropout(0.2)(x)
    outputs = tfl.Dense(10, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)

    return model


In [9]:
model = digits_model()

## Training

In [12]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

In [14]:
model.fit(X_train, y_train, batch_size=32, epochs=2)

Epoch 1/2
56/56 [==============================] - 48s 858ms/step - loss: 0.2759 - accuracy: 0.9018
Epoch 2/2
56/56 [==============================] - 49s 866ms/step - loss: 0.2696 - accuracy: 0.9064


## Fine tuning

### We will tune the last 5 layers of the resNet50 model

In [15]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 100, 100, 3)]     0         
_________________________________________________________________
tf.__operators__.getitem_1 ( (None, 100, 100, 3)       0         
_________________________________________________________________
tf.nn.bias_add_1 (TFOpLambda (None, 100, 100, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, 4, 4, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2049

#### Unfreeze last 5 layers

In [26]:
base_model = model.layers[3]
base_model.trainable = False

for layer in base_model.layers[-5:0]:
    layer.trainable = True

In [27]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

#### Fine-tuning

In [33]:
model.fit(X_train, y_train, batch_size=32, epochs=5)

Epoch 1/5
56/56 [==============================] - 52s 926ms/step - loss: 0.1994 - accuracy: 0.9427
Epoch 2/5
56/56 [==============================] - 49s 880ms/step - loss: 0.1973 - accuracy: 0.9410
Epoch 3/5
56/56 [==============================] - 49s 878ms/step - loss: 0.1944 - accuracy: 0.9512
Epoch 4/5
56/56 [==============================] - 50s 889ms/step - loss: 0.1931 - accuracy: 0.9472
Epoch 5/5
56/56 [==============================] - 49s 869ms/step - loss: 0.1962 - accuracy: 0.9359


## Testing

In [35]:
model.evaluate(X_train, y_train)

56/56 [==============================] - 52s 937ms/step - loss: 0.1426 - accuracy: 0.9739


[0.14264599978923798, 0.9738932847976685]

In [34]:
model.evaluate(X_valid, y_valid)

10/10 [==============================] - 10s 1s/step - loss: 0.3512 - accuracy: 0.8933


[0.3512345850467682, 0.8933333158493042]

## Save the model

In [36]:
model.save('../models/resNet.h5')